In [3]:
import torchvision,torch,time
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential

dataset_transform = torchvision.transforms.ToTensor()
train_data = torchvision.datasets.FashionMNIST(root="../dataset", train=True, transform=dataset_transform, download=False)
test_data = torchvision.datasets.FashionMNIST(root="../dataset", train=False, transform=dataset_transform, download=False)
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)
# length 长度
train_data_size = len(train_data)
test_data_size = len(test_data)
print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))

训练数据集的长度为：60000
测试数据集的长度为：10000


In [4]:
# 创建网络模型    
class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        self.model1 = Sequential(
            Conv2d(1, 32, 5, padding=4),
            MaxPool2d(2),
            Conv2d(32, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 64, 5, padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )

    def forward(self, x):
        x = self.model1(x)
        return x
    
    
tudui = Tudui()
tudui = tudui.cuda()
loss_fn = nn.CrossEntropyLoss() # 交叉熵损失函数
loss_fn = loss_fn.cuda()
learning_rate = 1e-2
optimizer = torch.optim.SGD(tudui.parameters(), lr=learning_rate)

total_train_step = 0    # 训练次数
total_test_step = 0     # 测试次数
epoch = 10              # 训练轮数
writer = SummaryWriter("../logs/hello_train")

In [6]:
last_time = time.time()
for i in range(epoch):
    print("-------第 {} 轮训练开始-------".format(i+1))
    # 训练
    tudui.train() # 对某些特定的层需要此句
    for data in train_dataloader:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = tudui(inputs)
        loss = loss_fn(outputs, labels)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            print("训练次数：{}, Loss: {}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    # 测试
    tudui.eval() # 对某些特定的层需要此句
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            imgs = imgs.cuda()
            targets = targets.cuda()
            outputs = tudui(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (outputs.argmax(1) == targets).sum() # argmax(0)竖着看，argmax(1)横着看
            total_accuracy = total_accuracy + accuracy

    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step = total_test_step + 1

    # torch.save(tudui, "../models/tudui_{}.pth".format(i))
    torch.save(tudui.state_dict(), "../models/hello/tudui_{}.pth".format(i))
    delta_time = time.time() - last_time
    last_time = time.time()
    print("模型已保存，耗时：{}".format(delta_time))

writer.close()

-------第 1 轮训练开始-------
训练次数：1000, Loss: 0.9991239905357361
训练次数：1100, Loss: 0.4827116131782532
训练次数：1200, Loss: 0.740709125995636
训练次数：1300, Loss: 0.6303516626358032
训练次数：1400, Loss: 0.7102474570274353
训练次数：1500, Loss: 0.43740758299827576
训练次数：1600, Loss: 0.6513292193412781
训练次数：1700, Loss: 0.7973306775093079
训练次数：1800, Loss: 0.4164527654647827
整体测试集上的Loss: 94.52657759189606
整体测试集上的正确率: 0.7795000076293945
模型已保存，耗时：6.36695671081543
-------第 2 轮训练开始-------
训练次数：1900, Loss: 0.5950825214385986
训练次数：2000, Loss: 0.7732735872268677
训练次数：2100, Loss: 0.5609692335128784
训练次数：2200, Loss: 0.6482155919075012
训练次数：2300, Loss: 0.5245014429092407
训练次数：2400, Loss: 0.7028173804283142
训练次数：2500, Loss: 0.5135456919670105
训练次数：2600, Loss: 0.5213502645492554
训练次数：2700, Loss: 0.537742555141449
训练次数：2800, Loss: 0.41852831840515137
整体测试集上的Loss: 83.77709713578224
整体测试集上的正确率: 0.8041999936103821
模型已保存，耗时：6.033871650695801
-------第 3 轮训练开始-------
训练次数：2900, Loss: 0.6056432723999023
训练次数：3000, Loss: 0.819864988327